In [3]:
from transformers import AutoTokenizer, AutoModel
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch
from torch import nn

import pandas as pd

In [4]:
model = AutoModel.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

In [5]:
original_sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু"
fake_sentence = "আপনি হতাশ কারণ  আমার জন্য অনেক কিছু করেছেন।"
fake_sentence = normalize(fake_sentence) # this normalization step is required before tokenizing the text
original_sentence = normalize(original_sentence)

In [6]:
fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode_plus(fake_sentence, return_tensors="pt")
# fake_inputs

original_inputs = tokenizer.encode_plus(original_sentence, return_tensors="pt")

In [7]:
discriminator_outputs = model(fake_inputs['input_ids'],attention_mask= fake_inputs['attention_mask'])#.logits
discriminator_outputs2 = model(original_inputs['input_ids'],attention_mask= original_inputs['attention_mask'])#.logits
# predictions = torch.round((torch.sign(discriminator_outputs) + 1) / 2)

In [8]:
input1 = discriminator_outputs['last_hidden_state'][:,0,:]
input2 = discriminator_outputs2['last_hidden_state'][:,0,:]

In [9]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [33]:
output = cos(input1, input2)
output

tensor([0.8526], grad_fn=<SumBackward1>)

In [10]:
df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)

/tmp/ipykernel_79845/2331687903.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
/tmp/ipykernel_79845/2331687903.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)


In [14]:
def consolidate(gdf):
    captions = gdf['caption'].to_list()
    return captions

df1 = df_train.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})
df2 = df_test.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})

In [15]:
df1.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)
df2.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)

df = pd.concat([df1, df2], axis=0)
df = df.reset_index(drop=True)

In [16]:
df.head()

,image_name,captions
0,1.jpg,"[একটি পার্কে একটি লেক আছে ।, লেকের উপর একটি ব্..."
1,2.jpg,"[একজন মানুষ মাথায় খড় নিয়ে হেটে যাচ্ছে ।, এক..."
2,3.jpg,"[টেবিলের উপর একটি চশমা আছে ।, টেবিলের উপর একটি..."
3,4.jpg,"[একটি শিশু দেখা যাচ্ছে ।, একটি শিশু গ্রামের ছো..."
4,5.jpg,"[একটি চায়ের কাপ দেখা যাচ্ছে ।, চায়ের কাপে উপ..."


In [19]:
df = pd.read_csv("dataset/bnature_captions.csv")
df.head()

,image_name,captions
0,1.jpg,"['গ্রামে হাঁটা দুই শিশু।', 'দুই শিশু গ্রামে হা..."
1,2.jpg,"['কিছু লোক নদী পার করছে।', 'কিছু লোক বিল পার ক..."
2,3.jpg,"['এক কৃষক ক্ষেতে কাজ করছে।', 'এক কৃষক ধান ক্ষে..."
3,4.jpg,"['কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।', '..."
4,5.jpg,"['পাঁচটি নারী এক জায়গায় চলছে।', 'পাঁচটি শিশু..."


In [18]:
import itertools

In [38]:
def top_3_captions(captions: list):
    """
    top 3 being the max length string, minimum length string and the string which has maximum cosine similarity index with 
    both of the max and min length string

    Args:
        captions (list): list of 5 captions from the bornon dataset for each image

    Returns:
        _type_: list of top 3 captions
    """
    cap_discriminator_outputs = []
    cap_inputs = []
    cap_final = []
    
    with torch.no_grad():
    # cap_final.append(tokenizer.encode_plus(normalize(max(captions, key=len)), return_tensors="pt"))
    # cap_final.append(tokenizer.encode_plus(normalize(min(captions, key=len)), return_tensors="pt"))
        cap_final.append(normalize(max(captions, key=len)))  
        cap_final.append(normalize(min(captions, key=len)))  
        
        for i in range(len(captions)):
            normalised = normalize(captions[i])
            cap_inputs.append(tokenizer.encode_plus(normalised, return_tensors="pt"))
        
        for i in range(len(cap_inputs)):
            cap_discriminator_outputs.append(model(cap_inputs[i]['input_ids'],attention_mask= cap_inputs[i]['attention_mask'])['last_hidden_state'][:,0,:])
        
        max_len_str = tokenizer.encode_plus(cap_final[0], return_tensors="pt")
        min_len_str = tokenizer.encode_plus(cap_final[1], return_tensors="pt")
        
        
        max_len_str = model(max_len_str['input_ids'],attention_mask= max_len_str['attention_mask'])['last_hidden_state'][:,0,:]
        min_len_str = model(min_len_str['input_ids'],attention_mask= min_len_str['attention_mask'])['last_hidden_state'][:,0,:]
        
        
        # print(max_len_str.shape)
        
        max_val = -1
        idx = 0
        for i in range(len(cap_discriminator_outputs)):
            # print(i)
            if cos(max_len_str, cap_discriminator_outputs[i]) > 0.99 or cos(min_len_str, cap_discriminator_outputs[i]) > 0.99:
                # print(captions[i], cap_final[0], cap_final[1])
                print("max: ", cos(max_len_str, cap_discriminator_outputs[i]), "min: ", cos(min_len_str, cap_discriminator_outputs[i]))
                
            else:
                val1 = cos(max_len_str, cap_discriminator_outputs[i])
                val2 = cos(min_len_str, cap_discriminator_outputs[i])
                
                # print(captions[i])
                # print(captions[i], cap_final[0], cap_final[1])
                
                if val1+val2 > max_val:
                    max_val = val1+val2
                    idx = i
                    
        cap_final.append(normalize(captions[idx]))
        
    return (cap_final)
    

In [39]:
df['top_captions'] = None
from tqdm import notebook as nb
import ast

In [40]:
for idx, row in (df.iterrows()):
    # caption_list = row['captions'].split(',')
    caption_list = ast.literal_eval(row['captions'])
    
    outs = top_3_captions(captions=caption_list)
    # df.loc[idx, 'top_captions'] = ',\n '.join(map(str, outs))
    
    df.loc[df['image_name'] == row['image_name'], 'top_captions'] = ',\n '.join(map(str, outs))
    

max:  tensor([1.0000]) min:  tensor([0.9101])
max:  tensor([0.9101]) min:  tensor([1.0000])
max:  tensor([0.9774]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.9774])
max:  tensor([0.9774]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.8934])
max:  tensor([0.8934]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.7002])
max:  tensor([0.7002]) min:  tensor([1.0000])
max:  tensor([1.]) min:  tensor([0.6663])
max:  tensor([0.6663]) min:  tensor([1.0000])
max:  tensor([0.7681]) min:  tensor([1.])
max:  tensor([1.]) min:  tensor([0.7681])
max:  tensor([0.8426]) min:  tensor([1.])
max:  tensor([1.]) min:  tensor([0.8426])
max:  tensor([1.]) min:  tensor([0.8554])
max:  tensor([0.8554]) min:  tensor([1.])
max:  tensor([0.8920]) min:  tensor([1.0000])
max:  tensor([1.0000]) min:  tensor([0.8920])
max:  tensor([0.9707]) min:  tensor([1.])
max:  tensor([1.0000]) min:  tensor([0.9707])
max:  tensor([0.7908]) min:  tensor([1.])
max:  tensor([1.]) min: 

In [41]:
df.head(10)

,image_name,captions,top_captions
0,1.jpg,"['গ্রামে হাঁটা দুই শিশু।', 'দুই শিশু গ্রামে হা...","দুই শিশু গ্রামে হাঁটছে।,\n দুই শিশু খেলা করে।,..."
1,2.jpg,"['কিছু লোক নদী পার করছে।', 'কিছু লোক বিল পার ক...","অনেকগুলো লোক নদী পার করছে।,\n কিছু লোক নদী পার..."
2,3.jpg,"['এক কৃষক ক্ষেতে কাজ করছে।', 'এক কৃষক ধান ক্ষে...","রোদ এর নিচে একটা লোক দাঁড়িয়ে আছে।,\n এক কৃষক..."
3,4.jpg,"['কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।', '...","কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।,\n সব..."
4,5.jpg,"['পাঁচটি নারী এক জায়গায় চলছে।', 'পাঁচটি শিশু...","পাঁচটি নারী এক জায়গায় চলছে।,\n পাঁচটি মেয়ে ..."
5,6.jpg,"['কিছু লোক পাহাড়ে বসে আছে।', 'কিছু লোক বসে আছ...","রোদ এর নিছে কিছু লোক বসে আছে।,\n কিছু লোক বসে ..."
6,7.jpg,"['নদীর পানিতে কিছু মানুষ।', 'বস্তা নিয়ে কিছু গ...","বস্তা নিয়ে কিছু গ্রামের মানুষ নদী পার হয়।,\n..."
7,8.jpg,"['পাঁচটি মহিলা একসাথে বসে আছে।', 'পাঁচটি মহিলা...","শাড়ী পরা পাঁচটি মহিলা কাজ করছে।,\n পাঁচটি মহি..."
8,9.jpg,"['মানুষ আকর্ষণ সঙ্গে ধান কাটা দেখছে।', 'একটি ল...",অনেকগুলো মানুষ ধান কাটার যন্ত্র দিয়ে ধান কাটা...
9,10.jpg,"['কিছু শিশু বালির মধ্যে খেলা করছে।', 'কিছু শিশ...","কিছু শিশু বালির মধ্যে এলমেলভাবে দৌড়াচ্ছে।,\n ..."


In [42]:
df.to_csv("dataset/bnature_top_captioned.csv", index=False)

## getting the largest caption only

In [2]:
df = pd.read_csv("dataset/generated_chitron.csv")

In [13]:
def longest_captions(captions: list):
    """
    each list is sent to this function only to return the longest one
    """
    # cap_discriminator_outputs = []
    
    longest = max(captions, key=len)
    # print(longest)
    return longest
    

In [24]:
# df['longest_caption'] = None
df.head()

,filename,caption,Generated,longest_caption
0,1.png,['তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দ...,NaN,'একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে...
1,2.png,['জলাশয় এ একজন ছেলে মানুষ ও একটি বাচ্চা মানুষ ...,NaN,'২ জন মানুষ যার একজন পুরুষ একটি নৌকায় দাড়িয়ে ...
2,3.png,"['অনেক মেয়ে মানুষ বসে আছে।', 'একটি নীল জামা পর...",NaN,'একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর ...
3,4.png,"['অনেক মানুষ একসাথে বসে কাজ করছে।', '২ টি ছোট...",NaN,'২ টি ছোট ছেলে একজন শার্ট প্যান্ট দাড়িয়ে চে...
4,5.png,"['ছয় জন মানুষ দাড়িয়ে আছে।', '৬ জন মানুষ এলোমেল...",NaN,"'৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ ..."


In [23]:
for idx, row in (df.iterrows()):
    
    string = row['caption']
    # print(s)
    list_of_strings = [s.strip("',") for s in string[1:-1].split("',")]
    # print(type(list_of_strings))
    outs = longest_captions(captions=list_of_strings)
    
    df.loc[df['filename'] == row['filename'], 'longest_caption'] = outs #',\n '.join(map(str, outs))
    # break

 'একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে একটি বেত নিয়ে পিটানোর ভাব দেখাচ্ছে আর ছোট একটি মেয়ে পিছনে ব্যাগ নিয়ে বসে কাঁদছে। 
 '২ জন মানুষ যার একজন পুরুষ একটি নৌকায় দাড়িয়ে চালাচ্ছে  সাদা লুঙ্গী ছাই রঙের গেঞ্জি পরে আরেকজন ছোট ছেলে পার থেকে ধাক্কা দিচ্ছে নউকাতিকে যার পরনে ফুল প্যান্ট খালি গা। 
 'একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর দিকে তাকিয়ে আছে এবং পিছনে  তার দিকে বসে শারি পরে তাকিয়ে আছে অনেকগুলো মহিলা। 
 '২ টি  ছোট ছেলে একজন শার্ট প্যান্ট  দাড়িয়ে চেয়ে আছে আরেকজন বসে গার্মেন্টস এ কাজ করছে নীল  শার্ট পরে তাদের পিছনে অনেকগুলো মহিলা বসে দাড়িয়ে কাজ করছে।  
 '৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ জন ছেলে ৪ জন পুরুষ,  তাদের একজন লুঙ্গী পরে দাড়িয়ে আছে। 
 'একটি মহিলা হালকা পানির উপরে দাড়িয়ে আছে শারি পরে, মহিলার মুখ ডানদিকে ঘুরানো, পানির রং হলুদ দেখাচ্ছে। 
 'একজন পুরুষ একটি কালো পাঞ্জাবী পায়জামা পরে  একটি ঢুল বাজাচ্ছে চুল এলোমেলো করে যার পিছনে কয়েকজন মহিলা , মেয়ে ঝাপসা দেখা যায় দাড়িয়ে আছে। 
 '৩ জন মানুষ চাতালে ধান নাড়া দিচ্ছে যার মাঝে একজন পুরুষ একটি কালো ছাতা এবং সাদা গেঞ্জি, 

In [25]:
df.isna().sum()

filename              0
caption               0
Generated          8669
longest_caption       0
dtype: int64

In [26]:
df.to_csv("dataset/generated_chitron.csv", index =False)

In [3]:
import transformers as hf
hf.utils.logging.set_verbosity_error()
print(hf.__version__)

4.40.2


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [ ]:
input_sentence = ""
input_ids = tokenizer(normalize(input_sentence), return_tensors="pt").input_ids
generated_tokens = model.generate(input_ids)
decoded_tokens = tokenizer.batch_decode(generated_tokens)[0]

print(decoded_tokens)